In [ ]:
pip install torch transformers datasets sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import json, ast, torch, random
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification
    # AdamW,
)
from sklearn.metrics import accuracy_score
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
pip install datasets

  Using cached datasets-3.5.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+c

In [ ]:
pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 33.8 MB/s eta 0:00:00


In [ ]:
import os
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForMaskedLM
from datasets import load_dataset
import torch

# Load teacher model (BioGPT)
# teacher_model_name = "microsoft/biogpt"
teacher_model_name = "dmis-lab/biobert-base-cased-v1.2"
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
teacher_model = AutoModelForMaskedLM.from_pretrained(teacher_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Load a small slice of PubMedQA dataset
# dataset = load_dataset("AdaptLLM/medicine-tasks", "PubMedQA", split="test[:20]")  # only first 10 samples

README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

test.json:   0%|          | 0.00/1.69M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PubMedQA_cleaned.json to PubMedQA_cleaned.json


In [ ]:
import pandas as pd

QA_data = pd.read_json('PubMedQA_cleaned.json')

In [ ]:
with open('PubMedQA_cleaned.json', "r") as f:
    raw_data = json.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Capstone/PubMedQA_cleaned.json'

import pandas as pd

QA_data = pd.read_json(file_path)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
QA_data

,id,context,question,options,gold_index
0,0,(Objective) We evaluated the usefulness of a s...,A short stay or 23-hour ward in a general and ...,"[No, Maybe, Yes]",2
1,1,(Methods) The records of 465 patients with an ...,Amblyopia: is visual loss permanent?,"[No, Maybe, Yes]",0
2,2,(Background) Radiotherapy reduces local recurr...,Does radiotherapy of the primary rectal cancer...,"[No, Maybe, Yes]",2
3,3,(Background) Pterygium is a disease of unknown...,Human papillomavirus and pterygium. Is the vir...,"[No, Maybe, Yes]",1
4,4,(Purpose) Reconstructing the natural joint lin...,Assessing joint line positions by means of the...,"[No, Maybe, Yes]",2
...,...,...,...,...,...
995,995,"(Background) ""America's Best Hospitals,"" an in...","Do ""America's Best Hospitals"" perform better f...","[No, Maybe, Yes]",2
996,996,(Background) Some patients with suspected comm...,The clinical significance of bile duct sludge:...,"[No, Maybe, Yes]",0
997,997,(Objective) To examine longitudinal patterns i...,Does obesity predict knee pain over fourteen y...,"[No, Maybe, Yes]",2
998,998,(Objectives) To assess Internet use amongst yo...,Can the Internet be used to improve sexual hea...,"[No, Maybe, Yes]",1


In [ ]:
sample = QA_data.head(3).tail(1)

In [ ]:
sample

,id,context,question,options,gold_index
2,2,(Background) Radiotherapy reduces local recurr...,Does radiotherapy of the primary rectal cancer...,"[No, Maybe, Yes]",2


In [ ]:
sample.loc[:,'question'].values[0]

'Does radiotherapy of the primary rectal cancer affect prognosis after pelvic exenteration for recurrent rectal cancer?'

In [ ]:
def build_prompt(input):
    input_context = input.loc[:,'context'].values[0]
    input_question = input.loc[:,'question'].values[0]
    prompt = f"""Read an abstract from a PubMed paper and answer the question: {input_context}

Question: {input_question}
Instruction: Return ONLY a confidence score over the three options ['No', 'Maybe', 'Yes'], DO NOT include any text output.
Format your answer strictly as: [prob_yes, prob_maybe, prob_no], where all numbers are between 0 and 1 and sum up to 1.
Example: [0.1, 0.2, 0.7]
"""
    return prompt

In [ ]:
# Example usage:
prompt_text = build_prompt(sample)
print(prompt_text)

Read an abstract from a PubMed paper and answer the question: (Background) Radiotherapy reduces local recurrence rates but is also capable of short- and long-term toxicity. It may also render treatment of local recurrence more challenging if it develops despite previous radiotherapy.
(Objective) This study examined the impact of radiotherapy for the primary rectal cancer on outcomes after pelvic exenteration for local recurrence.
(Design) We conducted a retrospective review of exenteration databases.
(Setting) The study took place at a quaternary referral center that specializes in pelvic exenteration.
(Patients) Patients referred for pelvic exenteration from October 1994 to November 2012 were reviewed. Patients who did and did not receive radiotherapy as part of their primary rectal cancer treatment were compared.
(Main outcome measures) The main outcomes of interest were resection margins, overall survival, disease-free survival, and surgical morbidities.
(Results) There were 108 pat

In [ ]:
teacher_model = teacher_model.to("cuda")

In [ ]:
# # Load teacher tokenizer for Causal models
# teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

# # 2. Set pad_token if not already set
# teacher_tokenizer.pad_token = teacher_tokenizer.eos_token

In [ ]:
inputs = teacher_tokenizer(
    prompt_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
).to(device = "cuda")

# Teacher inference
with torch.no_grad():
    teacher_logits = teacher_model(**inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: The size of tensor a (523) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
teacher_logits

MaskedLMOutput(loss=None, logits=tensor([[[-2.4603,  1.5772, -1.9107,  ..., -5.3926, -5.7133, -5.2893],
         [-1.6908, -2.0407,  4.0809,  ..., -5.3818, -4.2740, -4.5042],
         [-3.6311, -3.2828,  5.0152,  ..., -5.1565, -4.5508, -4.4634],
         ...,
         [-8.9352,  6.4139, -5.1730,  ..., -7.6896, -7.6727, -7.5745],
         [-1.8795,  2.7819, -0.0720,  ..., -4.1920, -4.7499, -4.1765],
         [-4.8602, -1.3108, -3.8114,  ..., -5.4039, -5.6652, -5.4817]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [ ]:

# 2. Tokenize it
#    return_tensors="pt" makes PyTorch tensors
#    padding=True  / truncation=True ensures fixed length if you batch later
inputs = teacher_tokenizer(
    prompt_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
).to(device = "cuda")   # e.g. device = "cuda" if you’re on GPU

# 3. Generate from the model
teacher_out_ids = teacher_model.generate(
    **inputs,         # expands to input_ids=…, attention_mask=…
    max_new_tokens=50
)

# 4. Decode back to text
decoded = teacher_tokenizer.decode(teacher_out_ids[0], skip_special_tokens=True)
print(decoded)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


TypeError: The current model class (BertForMaskedLM) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].

In [ ]:
import transformers
print(transformers.__version__)

4.51.3


In [ ]:
import json
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# 1. Load your local PubMedQA cleaned dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Capstone/PubMedQA_cleaned.json'

with open(file_path, "r") as f:
    raw_data = json.load(f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 2. Tokenizer
model_name = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 3. Custom Dataset Class
class PubMedQADataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt_text = f"Article: {item['context']}\nQuestion: {item['question']}\nAnswer with: 'Yes', 'No', or 'Maybe'."
        encoding = self.tokenizer(
            prompt_text,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        encoding["labels"] = torch.tensor(item["gold_index"], dtype=torch.long)
        return encoding

full_dataset = PubMedQADataset(raw_data, tokenizer)


In [ ]:
# 4. Split into Train and Test
train_size = int(0.8 * len(full_dataset))  # 80% train, 20% test
test_size = len(full_dataset) - train_size
train_dataset, eval_dataset = random_split(full_dataset, [train_size, test_size])

In [ ]:
# 5. Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 6. Set Training Arguments
training_args = TrainingArguments(
    output_dir="./biobert_pubmedqa",
    eval_strategy="epoch",
    # eval_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    save_strategy="epoch",
    # save_steps=100,
    save_total_limit=1,
    logging_dir="./logs",
    learning_rate=2e-5,
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True
)

In [ ]:
# 7. Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=1)
    acc = (preds == torch.tensor(labels)).float().mean()
    return {"accuracy": acc.item()}

In [ ]:
# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# 9. Train!
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.914100,0.959138,0.555000
2,0.855500,0.937257,0.565000
3,0.764200,0.872656,0.690000
4,0.552700,0.980940,0.615000
5,0.589000,1.054525,0.635000


TrainOutput(global_step=500, training_loss=0.7660350723266601, metrics={'train_runtime': 176.1718, 'train_samples_per_second': 22.705, 'train_steps_per_second': 2.838, 'total_flos': 1052453670912000.0, 'train_loss': 0.7660350723266601, 'epoch': 5.0})

In [ ]:
prompt_text = f"Article: {sample.loc[:,'context'].values[0]}\nQuestion: {sample.loc[:,'question'].values[0]}\nAnswer with: 'Yes', 'No', or 'Maybe'."

In [ ]:
len(prompt_text)

1889

In [ ]:
# 10. Save best model
trainer.save_model("biobert_pubmedqa_best")
tokenizer.save_pretrained("biobert_pubmedqa_best")

('biobert_pubmedqa_best/tokenizer_config.json',
 'biobert_pubmedqa_best/special_tokens_map.json',
 'biobert_pubmedqa_best/vocab.txt',
 'biobert_pubmedqa_best/added_tokens.json',
 'biobert_pubmedqa_best/tokenizer.json')

In [ ]:
!zip -r biobert_pubmedqa_best.zip biobert_pubmedqa_best

from google.colab import files
files.download('biobert_pubmedqa_best.zip')

  adding: biobert_pubmedqa_best/ (stored 0%)
  adding: biobert_pubmedqa_best/model.safetensors (deflated 7%)
  adding: biobert_pubmedqa_best/vocab.txt (deflated 49%)
  adding: biobert_pubmedqa_best/tokenizer_config.json (deflated 74%)
  adding: biobert_pubmedqa_best/special_tokens_map.json (deflated 42%)
  adding: biobert_pubmedqa_best/config.json (deflated 52%)
  adding: biobert_pubmedqa_best/training_args.bin (deflated 51%)
  adding: biobert_pubmedqa_best/tokenizer.json (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 11. Load your saved model
model_path = "biobert_pubmedqa_best"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
def build_prompt(input):
    input_context = input.loc[:,'context'].values[0]
    input_question = input.loc[:,'question'].values[0]
    prompt = f"""Read an abstract from a PubMed paper and answer the question: {input_context}

Question: {input_question}
Instruction: Return ONLY a confidence score over the three options ['No', 'Maybe', 'Yes'], DO NOT include any text output.
Format your answer strictly as: [prob_yes, prob_maybe, prob_no], where all numbers are between 0 and 1 and sum up to 1.
"""
    return prompt

In [ ]:
# Example usage:
prompt_text = build_prompt(sample)
print(prompt_text)

Read an abstract from a PubMed paper and answer the question: (Background) Radiotherapy reduces local recurrence rates but is also capable of short- and long-term toxicity. It may also render treatment of local recurrence more challenging if it develops despite previous radiotherapy.
(Objective) This study examined the impact of radiotherapy for the primary rectal cancer on outcomes after pelvic exenteration for local recurrence.
(Design) We conducted a retrospective review of exenteration databases.
(Setting) The study took place at a quaternary referral center that specializes in pelvic exenteration.
(Patients) Patients referred for pelvic exenteration from October 1994 to November 2012 were reviewed. Patients who did and did not receive radiotherapy as part of their primary rectal cancer treatment were compared.
(Main outcome measures) The main outcomes of interest were resection margins, overall survival, disease-free survival, and surgical morbidities.
(Results) There were 108 pat

In [ ]:
teacher_model = model
teacher_model = teacher_model.to("cuda")

In [ ]:
inputs = tokenizer(
    prompt_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
).to(device = "cuda")

In [ ]:
# Teacher inference
with torch.no_grad():
    teacher_outputs = teacher_model(**inputs)
    logits = teacher_outputs.logits

In [ ]:
logits

tensor([[ 0.4568, -0.9682,  1.4074]], device='cuda:0')

In [ ]:
import torch.nn.functional as F

probs = F.softmax(logits, dim=-1)

print(probs)

tensor([[0.2613, 0.0628, 0.6759]], device='cuda:0')


Confidence Scores: No=0.261, Maybe=0.063, Yes=0.676
